In [2]:
%cd ..

/home/nikolay/Documents/Projects/Diploma/My repos/Pseudo_label_Grounding_DINO


/home/nikolay/Documents/Projects/Diploma/My repos/Pseudo_label_Grounding_DINO/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
import glob
import pandas as pd
from utils import AnnotationDF
import cv2
import matplotlib.pyplot as plt
import tqdm
import numpy as np
import torch
from metrics import COCO_annotation_loader, count_metrics

In [4]:
CLASSES = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 
    'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 
    'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

In [5]:
def get_IoU(bbox1, bbox2):
        intersection_h_max = min(bbox1[1] + (bbox1[3]/2), bbox2[1] + (bbox2[3]/2))
        intersection_h_min = max(bbox1[1] - (bbox1[3]/2), bbox2[1] - (bbox2[3]/2))
        intersection_w_max = min(bbox1[0] + (bbox1[2]/2), bbox2[0] + (bbox2[2]/2))
        intersection_w_min = max(bbox1[0] - (bbox1[2]/2), bbox2[0] - (bbox2[2]/2))   
        intersection_area = (intersection_h_max - intersection_h_min) * (intersection_w_max - intersection_w_min)
        union = bbox1[2]*bbox1[3] + bbox2[2]*bbox2[3] - intersection_area
        return round(intersection_area / union, 3)

In [6]:
class pandas_annotation_loader():
    def __init__(self, ann_file, filters=None):
        data = AnnotationDF()
        data.load_annotation(ann_file)
        self.df = data.df.copy()
        self.filenames = sorted(self.df.image_name.unique())
        self.filters = filters


    def __len__(self):
        return len(self.df.shape[0])
    
    

    def __get_from_dataframe(self, filename):
        data = self.df[self.df.image_name == filename].sort_values(by=['probability'], ascending=False)

        # Filter
        if self.filters is not None:
            for filter in self.filters:
                data = data[filter[0](data, **filter[1])]

        # classes
        class_indexes = data.loc[:,['class_index']].to_numpy()
        # confidence
        confidences = data.loc[:,['probability']].to_numpy()
        # BBOX in format xywh
        bbox_string2bbox = lambda line: np.array([float(x) for x in line[0].split(' ')])
        bboxes = np.array(list(map(bbox_string2bbox, data.loc[:,['bbox']].to_numpy())))
        if bboxes.shape[0] > 0:
            return np.concatenate((bboxes, confidences, class_indexes), axis=1)
        else: return np.array([])

    # Возвращает list[Detection] по индексу
    def __getitem__(self, idx):
        filename = self.filenames[idx]
        return filename, self.__get_from_dataframe(filename)
    
    # Возвращает list[Detection] по индексу
    def get_annotation_by_name(self, filename):
        if filename not in self.filenames:
            print('not in the list')
            return None
        else:
            return self.__get_from_dataframe(filename) 

In [8]:
def nms_filter(detections, iou_threshold):
    # BBOX in format xywh
    bbox_string2bbox = lambda line: np.array([float(x) for x in line.split(' ')])
    keep_list = []
    mask = []
    for _, detection in detections.iterrows():
        keep = True
        for sample in keep_list:
            if get_IoU(bbox_string2bbox(sample.bbox), 
                       bbox_string2bbox(detection.bbox) ) > iou_threshold:
                keep = False
                break
        keep_list.append(detection)
        mask.append(keep)
    return np.array(mask)

def nms_filter_one_class(detections, iou_threshold):
    # BBOX in format xywh
    bbox_string2bbox = lambda line: np.array([float(x) for x in line.split(' ')])
    keep_list = []
    mask = []
    for _, detection in detections.iterrows():
        keep = True
        for sample in keep_list:
            if get_IoU(bbox_string2bbox(sample.bbox), 
                       bbox_string2bbox(detection.bbox) ) > iou_threshold and (sample.class_index == detection.class_index):
                keep = False
                break
        keep_list.append(detection)
        mask.append(keep)
    return np.array(mask)

def dumb_probability_filter(detections, probability_threshold):
    mask = []
    for _, detection in detections.iterrows():
        keep = True
        if detection.probability <= probability_threshold:
            keep = False
        mask.append(keep)
    return np.array(mask)


nms_filter_args = {'iou_threshold':0.7}
# nms_filter_one_class_args = {'iou_threshold':0.5}
dumb_probability_filter_args = {'probability_threshold':0.4}

filters = [
    (nms_filter, nms_filter_args),
    # (nms_filter_one_class, nms_filter_one_class_args),
    (dumb_probability_filter, dumb_probability_filter_args)
]

In [10]:
test_COCO = 'datasets/VOC/labels/test'
test_pred = 'data/VOC/test'

val_COCO = 'datasets/VOC/labels/val'
val_pred = 'data/VOC/val'

train_COCO = 'datasets/VOC/labels/train'
train_pred = 'data/VOC/train'



ann_loader = COCO_annotation_loader(val_COCO)
det_loader = pandas_annotation_loader(val_pred, filters)

ap, ap_class = count_metrics(det_loader, ann_loader)

100%|██████████| 2510/2510 [01:37<00:00, 25.77it/s]


In [49]:
# data = AnnotationDF(CLASSES)
# data.load_annotation('data/VOC2007/train2007/df_annotations')

In [11]:
# data.df[data.df.image_name == '000026']

In [9]:
map50, map50_95 = ap[:, 0].mean(), ap.mean(1).mean()
print(map50, map50_95)

0.7717903974207231 0.6601822521523052


In [12]:
map50, map50_95 = ap[:, 0].mean(), ap.mean(1).mean()
print(map50, map50_95)

0.7814329608949306 0.6688116542898079


In [17]:
for name, ap_cl in zip (CLASSES, ap[:,-1]):
    print(f'{name} : {ap_cl}')

aeroplane : 0.3262481195756159
bicycle : 0.3149881272232537
bird : 0.3039502269707657
boat : 0.1831356527351193
bottle : 0.1322229864396249
bus : 0.3945570686103146
car : 0.2610466580059484
cat : 0.4442934885832623
chair : 0.12511102560322193
cow : 0.2621817861832509
diningtable : 0.1989741376497271
dog : 0.39797197249100624
horse : 0.42526975901531877
motorbike : 0.28461678734930995
person : 0.2432586988481904
pottedplant : 0.058938631787894685
sheep : 0.21142443885863849
sofa : 0.25700754425834577
train : 0.32497425564572635
tvmonitor : 0.1616634977993748


In [15]:
for name, ap_cl in zip (CLASSES, ap[:,-1]):
    print(f'{name} : {ap_cl}')

aeroplane : 0.21254799343939446
bicycle : 0.19671853887388674
bird : 0.19527639964760754
boat : 0.087011846423598
bottle : 0.09471990778151669
bus : 0.28263913187403433
car : 0.17212190791643567
cat : 0.2794418293701749
chair : 0.06071717194393426
cow : 0.15184847838808918
diningtable : 0.16099513885638356
dog : 0.26943889903029705
horse : 0.26480642772761154
motorbike : 0.14732393725706605
person : 0.152191872892028
pottedplant : 0.025829802010285303
sheep : 0.1073687360217109
sofa : 0.20218703670607296
train : 0.19873540567009604
tvmonitor : 0.09773978572894092
